In [2]:
import time

import numpy as np
from IPython.display import Image
from keras.callbacks import TensorBoard
from keras.layers import Dense
from keras.models import Sequential
from pyswarms.single.global_best import GlobalBestPSO
from pyswarms.utils.functions import single_obj as fx
from pyswarms.utils.plotters import plot_surface
from pyswarms.utils.plotters.formatters import Animator, Designer, Mesher
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import warnings
warnings.filterwarnings("ignore")

In [4]:
iris = load_iris()
X = iris['data']
y = iris['target']
names = iris['target_names']
feature_names = iris['feature_names']
enc = OneHotEncoder()
Y = enc.fit_transform(y[:, np.newaxis]).toarray()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, Y_train, Y_test = train_test_split(
    X_scaled, Y, test_size=0.5, random_state=2)
n_features = X.shape[1]
n_classes = Y.shape[1]

In [5]:
def create_custom_model(input_dim, output_dim, nodes, n=1, name='model'):
    model = Sequential(name=name)
    for i in range(n):
        model.add(Dense(nodes, input_dim=input_dim, activation='relu'))
    model.add(Dense(output_dim, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model


In [6]:
n_layers = 1
model = create_custom_model(n_features, n_classes,
                            4, n_layers)
model.summary()

start_time = time.time()
print('Model name:', model.name)
history_callback = model.fit(X_train, Y_train,
                             batch_size=5,
                             epochs=400,
                             verbose=0,
                             validation_data=(X_test, Y_test)
                             )
score = model.evaluate(X_test, Y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print("--- %s seconds ---" % (time.time() - start_time))

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 20        
                                                                 
 dense_1 (Dense)             (None, 3)                 15        
                                                                 
Total params: 35
Trainable params: 35
Non-trainable params: 0
_________________________________________________________________
Model name: model
3/3 [==============================] - 0s 2ms/step - loss: 0.1235 - accuracy: 0.9467
Test loss: 0.12348707765340805
Test accuracy: 0.9466666579246521
--- 16.798799991607666 seconds ---


In [10]:
def get_shape(model):
    weights_layer = model.get_weights()
    shapes = []
    for weights in weights_layer:
        shapes.append(weights.shape)
    return shapes
def set_shape(weights,shapes):
    new_weights = []
    index=0
    for shape in shapes:
        if(len(shape)>1):
            n_nodes = np.prod(shape)+index
        else:
            n_nodes=shape[0]+index
        tmp = np.array(weights[index:n_nodes]).reshape(shape)
        new_weights.append(tmp)
        index=n_nodes
    return new_weights

In [9]:
start_time = time.time()
def evaluate_nn(W, shape,X_train=X_train, Y_train=Y_train):
    results = []
    for weights in W:
        model.set_weights(set_shape(weights,shape))
        score = model.evaluate(X_train, Y_train, verbose=0)
        results.append(1-score[1])
    return results

shape = get_shape(model)
x_max = 1.0 * np.ones(35)
x_min = -1.0 * x_max
bounds = (x_min, x_max)
options = {'c1': 0.4, 'c2': 0.6, 'w': 0.4}
optimizer = GlobalBestPSO(n_particles=50, dimensions=35,
                          options=options, bounds=bounds)
cost, pos = optimizer.optimize(evaluate_nn, 20, X_train=X_train, Y_train=Y_train,shape=shape)
model.set_weights(set_shape(pos,shape))
score = model.evaluate(X_test, Y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print("--- %s seconds ---" % (time.time() - start_time))

2021-12-28 01:35:13,267 - pyswarms.single.global_best - INFO - Optimize for 20 iters with {'c1': 0.4, 'c2': 0.6, 'w': 0.4}
pyswarms.single.global_best: 100%|██████████|20/20, best_cost=0.0133
2021-12-28 01:36:02,424 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.013333320617675781, best pos: [ 0.11273284 -0.44608439 -0.42068295  0.75235432  0.31250255  0.37742352
  0.34469984  0.20821845  0.5365929  -0.09791934 -0.51470203  0.11139093
  0.67417634 -0.10178379 -0.2683085   0.0365008   0.55132816  0.67694028
  0.47503709  0.12789514 -0.09193214  0.22279358  0.78369814  0.40921187
  0.4462695  -0.21505084  0.44101792 -0.07249371  0.08583225 -0.0394122
  0.01298918 -0.40660378  0.01284558  0.39313813  0.05990736]


3/3 [==============================] - 0s 4ms/step - loss: 0.6957 - accuracy: 0.9600
Test loss: 0.6957109570503235
Test accuracy: 0.9599999785423279
--- 49.24260330200195 seconds ---


In [47]:
m = Mesher(func=fx.sphere)
pos_history = [pos[:, :2] for pos in optimizer.pos_history]
pos3d = m.compute_history_3d(pos_history)
# Assuming we already had an optimizer ready
my_animator = Animator(repeat=False)
my_designer = Designer(figsize=(6, 6))
animation = plot_surface(pos3d, animator=my_animator, designer=my_designer)
# %%
animation.save('pso.gif', writer='pillowwritter', fps=6, )
Image(url='pso.gif')

2021-12-27 18:03:22,946 - matplotlib.animation - WARNING - MovieWriter pillowwritter unavailable; using Pillow instead.
2021-12-27 18:03:22,948 - matplotlib.animation - INFO - Animation.save using <class 'matplotlib.animation.PillowWriter'>


In [2]:
from sklearn.model_selection import KFold

In [9]:
X = iris["data"]
y = iris["target"]

enc = OneHotEncoder()
Y = enc.fit_transform(y[:, np.newaxis]).toarray()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

inputs = X_scaled
targets = Y

num_folds = 5

kfold = KFold(n_splits=num_folds, shuffle=True)

fold_no = 1
accs_bp = []

x_max = 1.0 * np.ones(35)
x_min = -1.0 * x_max
bounds = (x_min, x_max)

for train, test in kfold.split(inputs, targets):
    model = create_custom_model(n_features, n_classes,
                                4, 1)
    shape = get_shape(model)
    history_callback = model.fit(inputs[train], targets[train],
                                batch_size=5,
                                epochs=400,
                                verbose=0,
                                )
    score = model.evaluate(inputs[test], targets[test])
    fold_no += 1
    accs_bp.append(score[1])
    

1/1 [==============================] - 0s 124ms/step - loss: 0.0223 - accuracy: 1.0000


1/1 [==============================] - 0s 133ms/step - loss: 0.1082 - accuracy: 0.9333


In [97]:
accs_pso=[]
for train, test in kfold.split(inputs, targets):
    options = {'c1': 0.4, 'c2': 0.4, 'w': 0.6}
    optimizer = GlobalBestPSO(n_particles=25, dimensions=35,
                            options=options, bounds=bounds)
    cost, pos = optimizer.optimize(evaluate_nn, 20, X_train=inputs[train], Y_train=targets[train],shape=shape, verbose=0)
    model.set_weights(set_shape(pos,shape))
    score = model.evaluate(inputs[test], targets[test])
    accs_pso.append(score[1])

1/1 [==============================] - 0s 24ms/step - loss: 0.7537 - accuracy: 0.8667


In [25]:
accs_bp
print("Accuracy {:.2f} +/- {:.2f}".format(np.average(accs_bp),np.std(accs_bp)))

Accuracy 0.96 +/- 0.02


In [29]:
accs_pso
print("Accuracy {:.2f} +/- {:.2f}".format(np.average(accs_pso),np.std(accs_pso)))

Accuracy 0.91 +/- 0.03
